# Лабораториска вежба 3
## Машинска визија 2018/2019
### Андреј Јанчевски 151003

In [1]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

In [2]:
def white_balance(img):
    result = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    avg_a = np.average(result[:, :, 1])
    avg_b = np.average(result[:, :, 2])
    result[:, :, 1] = result[:, :, 1] - (
        (avg_a - 128) * (result[:, :, 0] / 255.0) * 1.1)
    result[:, :, 2] = result[:, :, 2] - (
        (avg_b - 128) * (result[:, :, 0] / 255.0) * 1.1)
    result = cv2.cvtColor(result, cv2.COLOR_LAB2BGR)
    return result

In [3]:
def adaptive_histogram_equalization(img):
    img[:, :, 0] = cv2.equalizeHist(img[:, :, 0])
    img[:, :, 1] = cv2.equalizeHist(img[:, :, 1])
    img[:, :, 2] = cv2.equalizeHist(img[:, :, 2])
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    img[:, :, 0] = clahe.apply(img[:, :, 0])
    img[:, :, 1] = clahe.apply(img[:, :, 1])
    img[:, :, 2] = clahe.apply(img[:, :, 2])

### Почетна слика од камера

In [4]:
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 480)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 360)
while True:
    captured, img = cap.read()
    adaptive_histogram_equalization(img)
    if captured:
        cv2.imshow("Camera Image", img)
        c = cv2.waitKey(1)
        if c == ord('q'):
            cap.release()
            break
    else:
        cap.release()
        break
cv2.destroyAllWindows()

In [5]:
def getSkinPixels(img, draw_lines=False):
    img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(img_hsv)
    face_mask_hue = cv2.inRange(h, 0, 20) + cv2.inRange(h, 160, 180)
    face_mask_saturation = cv2.inRange(s, 48, 150)
    face_mask_value = cv2.inRange(v, 80, 255)
    face_mask = cv2.bitwise_and(face_mask_hue, face_mask_saturation)
    # face_mask = cv2.bitwise_and(face_mask, face_mask_value)
    face_mask = cv2.medianBlur(face_mask, 31)
    face_mask = cv2.morphologyEx(
        face_mask, op=cv2.MORPH_OPEN, kernel=(15, 15), iterations=2)
    h = cv2.bitwise_and(h, face_mask)
    s = cv2.bitwise_and(s, face_mask)
    v = cv2.bitwise_and(v, face_mask)
    img_hsv_masked = cv2.merge((h, s, v))
    img_result = cv2.cvtColor(np.uint8(img_hsv_masked), cv2.COLOR_HSV2BGR)
    if draw_lines:
        img_result = np.copy(img)
        face_mask, contours, _ = cv2.findContours(face_mask, cv2.RETR_LIST,
                                                  cv2.CHAIN_APPROX_SIMPLE)
        cnt = max(contours, key=lambda x: cv2.contourArea(x))
        cv2.drawContours(img_result, [cnt], 0, (0, 0, 255), 3)
        cv2.rectangle(img_result, (0, 0), (128, 24), (0, 0, 0), -1)
        cv2.putText(
            img_result,
            str(cv2.contourArea(cnt)), (0, 16),
            fontFace=cv2.FONT_HERSHEY_PLAIN,
            fontScale=1,
            color=(255, 255, 255))
        cnt_x1 = min(cnt, key=lambda x: x[0][0])[0][0]
        cnt_x2 = max(cnt, key=lambda x: x[0][0])[0][0]
        cnt_y1 = min(cnt, key=lambda x: x[0][1])[0][1]
        cnt_y2 = max(cnt, key=lambda x: x[0][1])[0][1]
        cv2.rectangle(img_result, (cnt_x1, cnt_y1), (cnt_x2, cnt_y2),
                      (255, 0, 0), 3)
        convex_hull = cv2.convexHull(cnt)
        cv2.drawContours(img_result, [convex_hull], 0, (0, 255, 0), 3)
    return img_result

### Пикселите со боја на кожа

In [6]:
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 480)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 360)
while True:
    captured, img = cap.read()
    if captured:
        img = white_balance(img)
        adaptive_histogram_equalization(img)
        img_skin = getSkinPixels(img)
        cv2.imshow("Skin Color Image", img_skin)
        c = cv2.waitKey(1)
        if c == ord('q'):
            cap.release()
            break
    else:
        cap.release()
        break
cv2.destroyAllWindows()

### Наоѓање на контури на лицето

In [7]:
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 480)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 360)
while True:
    captured, img = cap.read()
    if captured:
        img = white_balance(img)
        adaptive_histogram_equalization(img)
        img_skin = getSkinPixels(img, draw_lines=True)
        cv2.imshow("Skin Color Image With Contours", img_skin)
        c = cv2.waitKey(1)
        if c == ord('q'):
            cap.release()
            break
    else:
        cap.release()
        break
cv2.destroyAllWindows()